# Quantitative Trading System - Results Analysis

This notebook analyzes the complete output from the NIFTY 50 trading system including:
- Regime detection visualization
- Strategy performance metrics
- ML model performance
- Trade analysis and outlier detection

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Load data
features_df = pd.read_csv('../data/nifty_features_5min.csv')
trades_df = pd.read_csv('../results/detailed_trades.csv')
outlier_df = pd.read_csv('../results/outlier_trades.csv')

print(f"Features shape: {features_df.shape}")
print(f"Trades: {len(trades_df)}")
print(f"Outliers: {len(outlier_df)}")

## 1. Market Regimes Analysis

In [ ]:
# Regime distribution
features_df['timestamp'] = pd.to_datetime(features_df['timestamp'])

# Visualize regime change
fig, axes = plt.subplots(2, 1, figsize=(15, 8))

# Price chart
axes[0].plot(features_df['timestamp'], features_df['close'], label='Close Price', color='black', alpha=0.7)
axes[0].set_ylabel('Price')
axes[0].set_title('NIFTY Spot Price')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Regime changes
uptrend = features_df[features_df['regime_label'] == 1]
downtrend = features_df[features_df['regime_label'] == -1]
sideways = features_df[features_df['regime_label'] == 0]

axes[1].scatter(uptrend['timestamp'], uptrend['iv'], color='green', s=5, alpha=0.6, label='Uptrend')
axes[1].scatter(downtrend['timestamp'], downtrend['iv'], color='red', s=5, alpha=0.6, label='Downtrend')
axes[1].scatter(sideways['timestamp'], sideways['iv'], color='gray', s=5, alpha=0.6, label='Sideways')
axes[1].set_ylabel('IV')
axes[1].set_xlabel('Time')
axes[1].set_title('Market Regime (colored by IV)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../plots/regime_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

# Regime statistics
print("\n=== REGIME STATISTICS ===")
print(f"\nRegime Distribution:")
print(features_df['regime_label'].value_counts().sort_index())
print(f"\nRegime Duration (hours):")
regime_changes = (features_df['regime_label'] != features_df['regime_label'].shift()).sum()
avg_duration = len(features_df) / regime_changes * 5 / 60  # 5-min bars to hours
print(f"Average regime duration: {avg_duration:.2f} hours")

## 2. Strategy Performance

In [ ]:
# Trade statistics
print("\n=== BASELINE STRATEGY PERFORMANCE ===")
profitable_trades = len(trades_df[trades_df['pnl'] > 0])
total_trades = len(trades_df)
win_rate = profitable_trades / total_trades * 100

print(f"Total Trades: {total_trades}")
print(f"Winning Trades: {profitable_trades}")
print(f"Losing Trades: {total_trades - profitable_trades}")
print(f"Win Rate: {win_rate:.2f}%")
print(f"\nAvg Winning Trade: Rs. {trades_df[trades_df['pnl'] > 0]['pnl'].mean():.2f}")
print(f"Avg Losing Trade: Rs. {trades_df[trades_df['pnl'] < 0]['pnl'].mean():.2f}")
print(f"Total P&L: Rs. {trades_df['pnl'].sum():.2f}")
print(f"Profit Factor: {abs(trades_df[trades_df['pnl'] > 0]['pnl'].sum()) / abs(trades_df[trades_df['pnl'] < 0]['pnl'].sum()):.2f}")

# Distribution analysis
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# PnL distribution
axes[0].hist(trades_df['pnl'], bins=30, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].axvline(trades_df['pnl'].mean(), color='red', linestyle='--', label=f'Mean: {trades_df["pnl"].mean():.2f}')
axes[0].axvline(0, color='black', linestyle='-', alpha=0.5)
axes[0].set_xlabel('Trade PnL (Rs.)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Trade PnL Distribution')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Duration analysis
axes[1].hist(trades_df['duration'], bins=30, color='coral', edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Trade Duration (5-min candles)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Trade Duration Distribution')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../plots/trade_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Regime-Based Trade Analysis

In [ ]:
print("\n=== REGIME-BASED TRADE ANALYSIS ===")

# Win rate by regime
for regime in [-1, 0, 1]:
    regime_trades = trades_df[trades_df['regime'] == regime]
    if len(regime_trades) > 0:
        win_rate = len(regime_trades[regime_trades['pnl'] > 0]) / len(regime_trades) * 100
        avg_pnl = regime_trades['pnl'].mean()
        regime_name = {-1: 'Downtrend', 0: 'Sideways', 1: 'Uptrend'}[regime]
        print(f"\n{regime_name}:")
        print(f"  Trades: {len(regime_trades)}")
        print(f"  Win Rate: {win_rate:.2f}%")
        print(f"  Avg PnL: Rs. {avg_pnl:.2f}")
        print(f"  IV: {regime_trades['iv'].mean():.2f}%")

# Box plot by regime
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

regime_names = {-1: 'Downtrend', 0: 'Sideways', 1: 'Uptrend'}
trades_df['regime_name'] = trades_df['regime'].map(regime_names)

# PnL by regime
sns.boxplot(data=trades_df, x='regime_name', y='pnl', ax=axes[0], palette='Set2')
axes[0].axhline(0, color='red', linestyle='--', alpha=0.5)
axes[0].set_title('Trade PnL by Regime')
axes[0].set_ylabel('PnL (Rs.)')
axes[0].set_xlabel('Regime')
axes[0].grid(True, alpha=0.3, axis='y')

# IV by regime
sns.boxplot(data=trades_df, x='regime_name', y='iv', ax=axes[1], palette='Set2')
axes[1].set_title('Implied Volatility by Regime')
axes[1].set_ylabel('IV (%)')
axes[1].set_xlabel('Regime')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../plots/regime_trade_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Outlier Trade Analysis (3-Sigma)

In [ ]:
print("\n=== OUTLIER TRADE ANALYSIS ===")
print(f"\nTotal Outlier Trades: {len(outlier_df)}")
print(f"Percentage of Total Trades: {len(outlier_df) / len(trades_df) * 100:.2f}%")

if len(outlier_df) > 0:
    print(f"\nOutlier Characteristics:")
    print(f"Average PnL: Rs. {outlier_df['pnl'].mean():.2f}")
    print(f"Average Duration: {outlier_df['duration'].mean():.2f} candles")
    print(f"Average IV: {outlier_df['iv'].mean():.2f}%")
    print(f"Average Vega: {outlier_df['vega'].mean():.4f}")
    
    print(f"\nOutlier Regimes:")
    print(outlier_df['regime'].value_counts())
    
    print(f"\nComparison: Outlier vs Normal Trades")
    normal_trades = trades_df[trades_df['z_score'] <= 3]
    
    comparison = pd.DataFrame({
        'Outliers': [
            outlier_df['pnl'].mean(),
            outlier_df['duration'].mean(),
            outlier_df['iv'].mean(),
            outlier_df['vega'].mean()
        ],
        'Normal': [
            normal_trades['pnl'].mean(),
            normal_trades['duration'].mean(),
            normal_trades['iv'].mean(),
            normal_trades['vega'].mean()
        ]
    }, index=['Avg PnL', 'Avg Duration', 'Avg IV', 'Avg Vega'])
    
    print(comparison)

    # Visualize outliers
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Scatter: PnL vs Duration
    axes[0, 0].scatter(normal_trades['duration'], normal_trades['pnl'], alpha=0.5, label='Normal', s=50)
    axes[0, 0].scatter(outlier_df['duration'], outlier_df['pnl'], color='red', s=200, marker='*', label='Outliers')
    axes[0, 0].axhline(0, color='black', linestyle='--', alpha=0.3)
    axes[0, 0].set_xlabel('Duration (candles)')
    axes[0, 0].set_ylabel('PnL (Rs.)')
    axes[0, 0].set_title('Outlier Trades: PnL vs Duration')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # IV comparison
    axes[0, 1].hist(normal_trades['iv'], bins=20, alpha=0.6, label='Normal', edgecolor='black')
    axes[0, 1].hist(outlier_df['iv'], bins=10, alpha=0.6, label='Outliers', color='red', edgecolor='black')
    axes[0, 1].set_xlabel('IV (%)')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].set_title('IV Distribution: Normal vs Outliers')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Vega comparison
    axes[1, 0].hist(normal_trades['vega'], bins=20, alpha=0.6, label='Normal', edgecolor='black')
    axes[1, 0].hist(outlier_df['vega'], bins=10, alpha=0.6, label='Outliers', color='red', edgecolor='black')
    axes[1, 0].set_xlabel('Vega')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].set_title('Vega Distribution: Normal vs Outliers')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Z-score distribution
    axes[1, 1].hist(trades_df['z_score'], bins=30, color='steelblue', edgecolor='black', alpha=0.7)
    axes[1, 1].axvline(3, color='red', linestyle='--', linewidth=2, label='3-sigma threshold')
    axes[1, 1].axvline(-3, color='red', linestyle='--', linewidth=2)
    axes[1, 1].set_xlabel('Z-Score')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].set_title('Trade PnL Z-Score Distribution')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../plots/outlier_analysis_detailed.png', dpi=150, bbox_inches='tight')
    plt.show()

## 5. Feature Correlation Analysis

In [ ]:
# Correlation in trades
corr_features = ['pnl', 'duration', 'iv', 'vega']
correlation_matrix = trades_df[corr_features].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='RdBu_r', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Trade Features Correlation Matrix')
plt.tight_layout()
plt.savefig('../plots/trade_correlation_detailed.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n=== FEATURE CORRELATIONS ===")
print(correlation_matrix)

## 6. Key Insights Summary

In [ ]:
print("\n" + "="*60)
print("QUANTITATIVE TRADING SYSTEM - COMPREHENSIVE ANALYSIS")
print("="*60)

print("\n1. DATA OVERVIEW")
print(f"   Total Data Points: {len(features_df):,}")
print(f"   Time Period: {features_df['timestamp'].min()} to {features_df['timestamp'].max()}")
print(f"   Price Range: Rs. {features_df['close'].min():.2f} - Rs. {features_df['close'].max():.2f}")

print("\n2. REGIME DETECTION")
for regime in [-1, 0, 1]:
    regime_name = {-1: 'Downtrend', 0: 'Sideways', 1: 'Uptrend'}[regime]
    pct = len(features_df[features_df['regime_label'] == regime]) / len(features_df) * 100
    avg_return = features_df[features_df['regime_label'] == regime]['returns'].mean()
    print(f"   {regime_name}: {pct:.1f}% of time, Avg Return: {avg_return:.4f}")

print("\n3. STRATEGY RESULTS")
print(f"   Total Trades: {len(trades_df)}")
print(f"   Win Rate: {win_rate:.2f}%")
print(f"   Total P&L: Rs. {trades_df['pnl'].sum():.2f}")
print(f"   Avg Trade Duration: {trades_df['duration'].mean():.0f} candles (~{trades_df['duration'].mean() * 5 / 60:.1f} hours)")

print("\n4. OUTLIER EVENTS")
print(f"   Count: {len(outlier_df)} ({len(outlier_df) / len(trades_df) * 100:.2f}% of trades)")
if len(outlier_df) > 0:
    print(f"   Avg PnL: Rs. {outlier_df['pnl'].mean():.2f}")
    print(f"   Impact: {abs(outlier_df['pnl'].sum()) / abs(trades_df['pnl'].sum()) * 100:.1f}% of total profit")

print("\n5. RISK METRICS")
print(f"   Max Drawdown Trade: Rs. {trades_df['pnl'].min():.2f}")
print(f"   Max Profit Trade: Rs. {trades_df['pnl'].max():.2f}")
print(f"   Std Dev of Returns: {trades_df['pnl'].std():.2f}")

print("\n" + "="*60)